In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [52]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [377]:
train_labels = pd.read_csv('training_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('training_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [378]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [379]:
train = train_values.join(train_labels)
train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2.00
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2.00
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3.00


In [380]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [381]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [382]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,0,3.00,25.72
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,0,2.00,25.72
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,0,3.00,25.72
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,2.00,25.72
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,3.00,30.08


In [383]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [384]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,3.00,25.72,11601
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,2.00,25.72,24929
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,3.00,25.72,24929
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,2.00,25.72,24929
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,3.00,30.08,11601


In [385]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [386]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,3.00,25.72,11601,1
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,2.00,25.72,24929,1
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,3.00,25.72,24929,1
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,2.00,25.72,24929,1
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,3.00,30.08,11601,1


In [387]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [388]:
train_value = train['age']
train_data = train.iloc[:, train.columns != 'age']

In [389]:
train_data

,count_floors_pre_eq,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,6,5,1,1,0,0,0,0,0,...,0,0,0,0,0,0,3.00,25.72,11601,1
28830,2,8,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2.00,25.72,24929,1
94947,2,5,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,3.00,25.72,24929,1
590882,2,6,5,0,1,0,0,0,0,1,...,0,0,0,0,0,0,2.00,25.72,24929,1
201944,3,8,9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,3.00,30.08,11601,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599698,1,3,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,nan,17.61,11601,1
551307,1,6,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,nan,17.61,23042,1
11398,2,9,4,0,1,0,0,0,0,0,...,0,0,0,0,0,0,nan,25.72,21590,1


In [296]:
test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [297]:
test = test_values.join(test_labels,how="inner")

In [298]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

In [299]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [300]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

In [301]:
#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

In [302]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


test = test.drop_duplicates('age')
IDs = test[['age']].copy()
test = test.drop('age', axis=1)

In [305]:
test.head()

,count_floors_pre_eq,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,5,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,11,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,9,10,0,0,0,0,0,1,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,9,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
476605,3,8,6,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,30.07,128,1


In [310]:
#test_value = test['builindg_id']
test_data = test.iloc[:, test.columns != 'building_id']

In [311]:
test.head()

,count_floors_pre_eq,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,5,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,11,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,9,10,0,0,0,0,0,1,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,9,7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
476605,3,8,6,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,30.07,128,1


In [312]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [313]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [314]:
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [315]:
%%capture
modelCatBoost=CatBoostRegressor(iterations=250, depth=3, learning_rate=0.05)
modelCatBoost.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

0:	learn: 69.0456187	test: 71.6817446	best: 71.6817446 (0)	total: 9.08ms	remaining: 2.26s
1:	learn: 65.6876601	test: 68.1930354	best: 68.1930354 (1)	total: 17.7ms	remaining: 2.2s
2:	learn: 62.5325511	test: 64.9127226	best: 64.9127226 (2)	total: 26.3ms	remaining: 2.16s
3:	learn: 59.5006273	test: 61.7629983	best: 61.7629983 (3)	total: 34.6ms	remaining: 2.12s
4:	learn: 56.6191751	test: 58.7696701	best: 58.7696701 (4)	total: 43.3ms	remaining: 2.12s
5:	learn: 53.8868045	test: 55.9308930	best: 55.9308930 (5)	total: 51.6ms	remaining: 2.1s
6:	learn: 51.3211592	test: 53.2637745	best: 53.2637745 (6)	total: 60ms	remaining: 2.08s
7:	learn: 48.8645001	test: 50.7109809	best: 50.7109809 (7)	total: 68.5ms	remaining: 2.07s
8:	learn: 46.5239811	test: 48.2794252	best: 48.2794252 (8)	total: 77.3ms	remaining: 2.07s
9:	learn: 44.2958831	test: 45.9649698	best: 45.9649698 (9)	total: 85.7ms	remaining: 2.06s
10:	learn: 42.1820087	test: 43.7690566	best: 43.7690566 (10)	total: 94ms	remaining: 2.04s
11:	learn: 40.

109:	learn: 4.3355557	test: 4.4717387	best: 4.4717387 (109)	total: 1.06s	remaining: 1.35s
110:	learn: 4.2889621	test: 4.4242277	best: 4.4242277 (110)	total: 1.07s	remaining: 1.35s
111:	learn: 4.2559136	test: 4.3887995	best: 4.3887995 (111)	total: 1.08s	remaining: 1.34s
112:	learn: 4.2129583	test: 4.3471239	best: 4.3471239 (112)	total: 1.09s	remaining: 1.33s
113:	learn: 4.1679864	test: 4.3032339	best: 4.3032339 (113)	total: 1.1s	remaining: 1.32s
114:	learn: 4.1482088	test: 4.2715813	best: 4.2715813 (114)	total: 1.11s	remaining: 1.3s
115:	learn: 4.1207316	test: 4.2443221	best: 4.2443221 (115)	total: 1.12s	remaining: 1.3s
116:	learn: 4.0817845	test: 4.2062558	best: 4.2062558 (116)	total: 1.13s	remaining: 1.29s
117:	learn: 4.0638426	test: 4.1772877	best: 4.1772877 (117)	total: 1.14s	remaining: 1.27s
118:	learn: 4.0282235	test: 4.1424026	best: 4.1424026 (118)	total: 1.15s	remaining: 1.27s
119:	learn: 3.9972962	test: 4.1133288	best: 4.1133288 (119)	total: 1.16s	remaining: 1.25s
120:	learn: 3

209:	learn: 2.1226574	test: 2.1799928	best: 2.1799928 (209)	total: 2.07s	remaining: 394ms
210:	learn: 2.1127099	test: 2.1703450	best: 2.1703450 (210)	total: 2.08s	remaining: 384ms
211:	learn: 2.0961382	test: 2.1536120	best: 2.1536120 (211)	total: 2.09s	remaining: 374ms
212:	learn: 2.0822072	test: 2.1401719	best: 2.1401719 (212)	total: 2.1s	remaining: 365ms
213:	learn: 2.0716539	test: 2.1249133	best: 2.1249133 (213)	total: 2.11s	remaining: 354ms
214:	learn: 2.0624666	test: 2.1159939	best: 2.1159939 (214)	total: 2.12s	remaining: 345ms
215:	learn: 2.0560607	test: 2.1097276	best: 2.1097276 (215)	total: 2.13s	remaining: 336ms
216:	learn: 2.0407717	test: 2.0944469	best: 2.0944469 (216)	total: 2.15s	remaining: 327ms
217:	learn: 2.0316198	test: 2.0808369	best: 2.0808369 (217)	total: 2.17s	remaining: 318ms
218:	learn: 2.0025864	test: 2.0543096	best: 2.0543096 (218)	total: 2.18s	remaining: 309ms
219:	learn: 1.9940167	test: 2.0459924	best: 2.0459924 (219)	total: 2.2s	remaining: 300ms
220:	learn: 

In [316]:
preds = modelCatBoost.predict(X_train)
logloss = log_loss(y_train, preds)
print("logloss: %f" % (logloss))

ValueError: y_true and y_pred contain different number of classes 38, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [  0   5  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85
  90  95 100 105 110 115 120 125 130 140 145 150 160 165 170 175 190 195
 200 995]

In [320]:
%%capture
modelCatBoost.fit(train_data,train_value,cat_features=categorical_features_indices)
preds = modelCatBoost.predict(test)
entregable_CatBoost = IDs[['building_id']].copy()
entregable_CatBoost['age'] = preds

0:	learn: 69.8396585	total: 13.3ms	remaining: 3.3s
1:	learn: 66.4339327	total: 25.7ms	remaining: 3.19s
2:	learn: 63.2036013	total: 37.8ms	remaining: 3.11s
3:	learn: 60.1318760	total: 49.8ms	remaining: 3.06s
4:	learn: 57.2169975	total: 62ms	remaining: 3.04s
5:	learn: 54.4571057	total: 73.8ms	remaining: 3s
6:	learn: 51.8391887	total: 86ms	remaining: 2.98s
7:	learn: 49.3394830	total: 98ms	remaining: 2.96s
8:	learn: 46.9684147	total: 110ms	remaining: 2.95s
9:	learn: 44.7197707	total: 122ms	remaining: 2.94s
10:	learn: 42.5834123	total: 135ms	remaining: 2.93s
11:	learn: 40.5620446	total: 150ms	remaining: 2.98s
12:	learn: 38.6392615	total: 162ms	remaining: 2.96s
13:	learn: 36.8130674	total: 174ms	remaining: 2.94s
14:	learn: 35.0797825	total: 186ms	remaining: 2.91s
15:	learn: 33.4395528	total: 198ms	remaining: 2.89s
16:	learn: 31.8869575	total: 210ms	remaining: 2.88s
17:	learn: 30.4171570	total: 227ms	remaining: 2.93s
18:	learn: 29.0161233	total: 240ms	remaining: 2.92s
19:	learn: 27.6872912	to

168:	learn: 2.7746214	total: 2.56s	remaining: 1.23s
169:	learn: 2.7540994	total: 2.57s	remaining: 1.21s
170:	learn: 2.7400023	total: 2.59s	remaining: 1.19s
171:	learn: 2.7212829	total: 2.6s	remaining: 1.18s
172:	learn: 2.6807741	total: 2.62s	remaining: 1.16s
173:	learn: 2.6516239	total: 2.63s	remaining: 1.15s
174:	learn: 2.6250287	total: 2.64s	remaining: 1.13s
175:	learn: 2.5871787	total: 2.65s	remaining: 1.11s
176:	learn: 2.5736911	total: 2.67s	remaining: 1.1s
177:	learn: 2.5568574	total: 2.69s	remaining: 1.09s
178:	learn: 2.5388486	total: 2.71s	remaining: 1.08s
179:	learn: 2.5035705	total: 2.75s	remaining: 1.07s
180:	learn: 2.4782234	total: 2.79s	remaining: 1.06s
181:	learn: 2.4655505	total: 2.81s	remaining: 1.05s
182:	learn: 2.4487442	total: 2.83s	remaining: 1.04s
183:	learn: 2.4334645	total: 2.85s	remaining: 1.02s
184:	learn: 2.4218372	total: 2.88s	remaining: 1.01s
185:	learn: 2.3885129	total: 2.9s	remaining: 996ms
186:	learn: 2.3728943	total: 2.91s	remaining: 981ms
187:	learn: 2.3

KeyError: "None of [Index(['building_id'], dtype='object')] are in the [columns]"

240:	learn: 1.5660527	total: 3.65s	remaining: 136ms
241:	learn: 1.5596479	total: 3.66s	remaining: 121ms
242:	learn: 1.5480221	total: 3.67s	remaining: 106ms
243:	learn: 1.5298379	total: 3.69s	remaining: 90.7ms
244:	learn: 1.5220447	total: 3.7s	remaining: 75.5ms
245:	learn: 1.5130084	total: 3.71s	remaining: 60.4ms
246:	learn: 1.5021503	total: 3.72s	remaining: 45.2ms
247:	learn: 1.4961244	total: 3.74s	remaining: 30.1ms
248:	learn: 1.4877756	total: 3.75s	remaining: 15.1ms
249:	learn: 1.4805411	total: 3.76s	remaining: 0us


In [321]:
entregable_CatBoost.to_csv('CatBoost.csv', index=False)

NameError: name 'entregable_CatBoost' is not defined

In [322]:
modeloCB = CatBoostRegressor()
LogLoss = make_scorer(log_loss, greater_is_better = False)
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [323]:
parameters = {'iterations':[100, 250, 500] , 'depth':[2, 3, 5], 'learning_rate':[0.005, 0.05, 0.5]}
gscv = GridSearchCV(modeloCB, parameters, cv=5, scoring=LogLoss)

In [326]:
%%capture
gscv.fit(train_data, train_values, cat_features = categorical_features_indices)

ValueError: Found input variables with inconsistent numbers of samples: [133648, 167062]

In [325]:
print(gscv.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
%%capture
modeloCB = CatBoostRegressor(depth = 5, iterations = 100, learning_rate = 0.05)
modeloCB.fit(train_data, train_value, cat_features = categorical_features_indices)
result = modeloCB.predict(test)

entregable_CB = IDs[['building_id']].copy()
entregable_CB['age'] = result

In [ ]:
entregable_CB.to_csv('CB_Final.csv', index=False)